In [1]:
import pandas as pd

In [2]:
result_file = "test_result_w_succ_diff_nodes_gru_2025_04_10_11_28.csv"

In [3]:
EPS = 1

In [4]:
df = pd.read_csv(result_file)

df = df.rename(columns={"Dataset": "Graph", "Actual": "Rank"})

df.head()

,Graph,Rank,Predicted
0,star_graph_n7,0.0,0.000000
1,star_graph_n7,0.0,0.000000
2,star_graph_n7,0.0,0.000000
3,star_graph_n7,3.0,3.000129
4,star_graph_n7,3.0,3.084559


In [5]:
df['Difference'] = abs(df['Rank']-df['Predicted'])
df['Difference_sq'] = df['Difference']**2
# df.head()

In [6]:
df['EPS'] = df['Rank'].apply(lambda x: EPS if x==0.0 else 0.0)
# df

In [7]:
df['RE'] = df['Difference']/(df['Rank']+df['EPS'])
# df.head()

In [8]:
df

,Graph,Rank,Predicted,Difference,Difference_sq,EPS,RE
0,star_graph_n7,0.0,0.000000,0.000000,0.000000e+00,1.0,0.000000
1,star_graph_n7,0.0,0.000000,0.000000,0.000000e+00,1.0,0.000000
2,star_graph_n7,0.0,0.000000,0.000000,0.000000e+00,1.0,0.000000
3,star_graph_n7,3.0,3.000129,0.000129,1.676023e-08,0.0,0.000043
4,star_graph_n7,3.0,3.084559,0.084559,7.150218e-03,0.0,0.028186
...,...,...,...,...,...,...,...
5275,graph_powerlaw_cluster_graph_n7,3.0,3.105450,0.105450,1.111973e-02,0.0,0.035150
5276,graph_powerlaw_cluster_graph_n7,1.0,0.994429,0.005571,3.103480e-05,0.0,0.005571
5277,graph_powerlaw_cluster_graph_n7,1.0,0.975742,0.024258,5.884283e-04,0.0,0.024258
5278,graph_powerlaw_cluster_graph_n7,2.0,1.895229,0.104771,1.097697e-02,0.0,0.052386


In [9]:
df['RE'].sum()/len(df)

0.063806114323964

In [10]:
def group_by_table(df, groupby):
    values_count = df[groupby].value_counts().reset_index()
    values_count = values_count.rename(columns={"count": "Count"})
    re_sum = df.groupby(groupby)["RE"].sum().reset_index()
    mre_error = df.groupby(groupby)["RE"].mean().reset_index()
    mre_error = mre_error.rename(columns={"RE": "MRE"})
    mse_error = df.groupby(groupby)["Difference_sq"].mean().reset_index()
    mse_error = mse_error.rename(columns={"Difference_sq": "MSE"})

    return values_count, re_sum, mre_error, mse_error

# Group by Rank

In [11]:
groupby = ['Rank']

values_count, re_sum, mre_error, mse_error = group_by_table(df, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Count', 'RE'], ascending=[False, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_rank.csv", index=False)
df_result


,Rank,Count,RE,MRE,MSE
0,3.0,1686,114.164671,0.067713,0.086916
1,2.0,1666,129.809066,0.077917,0.055835
2,1.0,910,30.810708,0.033858,0.002279
3,4.0,701,54.466570,0.077698,0.157842
4,0.0,256,0.000000,0.000000,0.000000
5,5.0,61,7.645269,0.125332,0.526315


# Group by Dataset

In [12]:
groupby = ['Graph']

values_count, re_sum, mre_error, mse_error = group_by_table(df, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Count', 'RE'], ascending=[False, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_dataset.csv", index=False)
df_result


,Graph,Count,RE,MRE,MSE
0,graph_random_regular_graph_n7_d4,3907,237.892227,0.060889,0.072786
1,graph_powerlaw_cluster_graph_n7,1350,98.113601,0.072677,0.073642
2,star_graph_n7,23,0.890456,0.038715,0.025883


# Group by Dataset W/O Rank 0

In [13]:
groupby = ['Graph']

df_wo_0 = df[df['Rank']!=0]
values_count, re_sum, mre_error, mse_error = group_by_table(df_wo_0, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Count', 'RE'], ascending=[False, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_dataset_wo_rank0.csv", index=False)
df_result

,Graph,Count,RE,MRE,MSE
0,graph_random_regular_graph_n7_d4,3763,237.892227,0.063219,0.075571
1,graph_powerlaw_cluster_graph_n7,1256,98.113601,0.078116,0.079154
2,star_graph_n7,5,0.890456,0.178091,0.119060


# Group by Dataset and Rank

In [14]:
groupby = ['Graph', 'Rank']

values_count, re_sum, mre_error, mse_error = group_by_table(df, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Graph', 'Count'], ascending=[True, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_dataset_rank.csv", index=False)
df_result

,Graph,Rank,Count,RE,MRE,MSE
0,graph_powerlaw_cluster_graph_n7,2.0,455,51.686300,0.113596,0.107137
1,graph_powerlaw_cluster_graph_n7,1.0,349,12.858660,0.036844,0.002391
2,graph_powerlaw_cluster_graph_n7,3.0,311,23.375595,0.075163,0.090785
3,graph_powerlaw_cluster_graph_n7,4.0,130,9.175549,0.070581,0.133316
4,graph_powerlaw_cluster_graph_n7,0.0,94,0.000000,0.000000,0.000000
5,graph_powerlaw_cluster_graph_n7,5.0,11,1.017498,0.092500,0.388183
6,graph_random_regular_graph_n7_d4,3.0,1373,90.760847,0.066104,0.086160
7,graph_random_regular_graph_n7_d4,2.0,1210,78.001329,0.064464,0.036541
8,graph_random_regular_graph_n7_d4,4.0,571,45.291021,0.079319,0.163426
9,graph_random_regular_graph_n7_d4,1.0,560,17.232160,0.030772,0.001288
